In [ ]:
# grad-CAM을 찍어볼 데이터셋
target_dataset_name = "exp8" 
# Source only 모델 (No domain adaptation)
MODEL_PATH = "/root/dezsda/checkpoints/timm-mobilenet_v2-epoch=49-task=nexmon-val_loss=0.4483.ckpt"
# DANN 모델 (domain adaptation)
MODEL_PATH_DANN = {
    "exp2": "/root/dezsda/checkpoints/dann-mobilenet_v2-epoch=49-task=nexmon-val_loss_tgt=0.7519.ckpt",
    "exp3": "/root/dezsda/checkpoints/dann-mobilenet_v2-epoch=49-task=nexmon-val_loss_tgt=0.7335.ckpt",
    "exp4": "/root/dezsda/checkpoints/dann-mobilenet_v2-epoch=49-task=nexmon-val_loss_tgt=0.8891.ckpt",
    "exp5": "/root/dezsda/checkpoints/dann-mobilenet_v2-epoch=49-task=nexmon-val_loss_tgt=1.7659.ckpt",
    "exp6": "/root/dezsda/checkpoints/dann-mobilenet_v2-epoch=49-task=nexmon-val_loss_tgt=1.2919.ckpt",
    "exp7": "/root/dezsda/checkpoints/dann-mobilenet_v2-epoch=49-task=nexmon-val_loss_tgt=1.0723.ckpt",
    "exp8": "/root/dezsda/checkpoints/dann-mobilenet_v2-epoch=49-task=nexmon-val_loss_tgt=1.3905.ckpt",
}

In [ ]:
from module.etc import Flatten
from omegaconf import open_dict
from dataset.nexmon import NexmonDataset
from albumentations.pytorch.transforms import ToTensorV2
import cv2
# from skimage.transform import resize
import numpy as np
from torchvision import models
from grad_cam import GradCam,GuidedBackpropReLUModel,show_cams,get_cam,show_gbs,preprocess_image
from hydra import initialize_config_dir, compose
from pl_module.timm import LitTimm
from pl_module.dann import LitDANN
import albumentations as A
import torch.nn as nn
import matplotlib.pyplot as plt
import tqdm

In [ ]:
with initialize_config_dir(config_dir="/root/dezsda"):
    config = compose(config_name="common.yaml", overrides=["config=timm", "channels=3"])
    with open_dict(config):
        config.num_classes = 7
        config.num_test_sets = 1
# model = models.vgg19(pretrained=True)
pl_model = LitTimm.load_from_checkpoint(MODEL_PATH, config=config)
model = nn.Sequential(
    pl_model.model.feature_extractor,
    pl_model.model.global_pool,
    Flatten(),
    pl_model.model.head,
)

with initialize_config_dir(config_dir="/root/dezsda"):
    config = compose(config_name="common.yaml", overrides=["config=dann", "channels=3"])
    with open_dict(config):
        config.num_classes = 7
        config.num_test_sets = 1
        config.batch_size = 16
pl_model_dann = LitDANN.load_from_checkpoint(MODEL_PATH_DANN[target_dataset_name], config=config)
model_dann = nn.Sequential(
    pl_model_dann.model.feature_extractor,
    pl_model_dann.model.global_pool,
    Flatten(),
    pl_model_dann.model.head,
)

In [ ]:
# plt.rcParams["figure.figsize"] = (7, 5)
# transform = A.Compose([
#     # A.Resize(self.config.img_size, self.config.img_size),
#     # A.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5]),
#     A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#     ToTensorV2(),
# ])

In [ ]:
# test_dataset = NexmonDataset(
#     root=f"/shared/lorenzo/data-tubuki-cache/exp1-cwt",
#     fold=config.fold_no,
#     train=False,
#     transform=transform,
#     single_per_class=True,
# )

# grad_cam = GradCam(model=model, blob_name = 'features', target_layer_names=['18'], use_cuda=False)
# # img = cv2.imread('temp.jpg', 1)
# # img = np.float32(cv2.resize(img, (224, 224))) / 255
# # img = np.float32(img) / 255
# # inputs = preprocess_image(img)
# # If None, returns the map for the highest scoring category.
# # Otherwise, targets the requested index.
# for batch in test_dataset:
#     inputs, targets = batch
#     target_index = None
#     mask_dic = grad_cam(inputs.unsqueeze(0), target_index)

#     for layer, mask in mask_dic.items():
#         plt.imshow(inputs.permute(1,2,0))
#         cam = get_cam(mask)
#         plt.imshow(cam, alpha=0.7)
#         plt.title(test_dataset.classes[targets])
#         plt.show()
# # show_cams(inputs.permute(1,2,0), mask_dic, test_dataset.classes[targets])
# # gb_model = GuidedBackpropReLUModel(model=model, activation_layer_name = 'ReLU6', use_cuda=False)
# # show_gbs(inputs.unsqueeze(0), gb_model, target_index, mask_dic)

In [ ]:
# transform = A.Compose([
#     # A.Resize(self.config.img_size, self.config.img_size),
#     # A.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5]),
#     A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#     ToTensorV2(),
# ])
# test_dataset = NexmonDataset(
#     root=f"/shared/lorenzo/data-tubuki-cache/exp1-cwt",
#     fold=config.fold_no,
#     train=False,
#     transform=transform,
#     # single_per_class=True,
# )

# grad_cam = GradCam(model=model, blob_name = 'features', target_layer_names=['18'], use_cuda=True)
# # img = cv2.imread('temp.jpg', 1)
# # img = np.float32(cv2.resize(img, (224, 224))) / 255
# # img = np.float32(img) / 255
# # inputs = preprocess_image(img)
# # If None, returns the map for the highest scoring category.
# # Otherwise, targets the requested index.
# cam_by_label = {}
# for batch in tqdm.tqdm(test_dataset):
#     inputs, targets = batch
#     target_index = None
#     mask_dic = grad_cam(inputs.unsqueeze(0).to("cuda:0"), target_index)

#     for layer, mask in mask_dic.items():
#         # plt.imshow(inputs.permute(1,2,0))
#         cam = get_cam(mask)
#         cam = (cam > 200).astype(float)
#         # plt.imshow((cam > 200).astype(float))
#         label = test_dataset.classes[targets]
#         # plt.title(label)
#         # plt.show()
#         if label not in cam_by_label:
#             cam_by_label[label] = [cam]
#         else:
#             cam_by_label[label].append(cam)
            
# for k, v in cam_by_label.items():
#     plt.imshow(np.mean(v, axis=0))
#     plt.title(k)
#     plt.show()
# # show_cams(inputs.permute(1,2,0), mask_dic, test_dataset.classes[targets])
# # gb_model = GuidedBackpropReLUModel(model=model, activation_layer_name = 'ReLU6', use_cuda=False)
# # show_gbs(inputs.unsqueeze(0), gb_model, target_index, mask_dic)

In [ ]:
plt.rcParams["figure.figsize"] = (14, 10)

transform = A.Compose([
    # A.Resize(self.config.img_size, self.config.img_size),
    # A.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5]),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

test_dataset = NexmonDataset(
    root=f"/shared/lorenzo/data-tubuki-cache/{target_dataset_name}-cwt",
    fold=config.fold_no,
    train=False,
    transform=transform,
    single_per_class=True, # 각 클래스별로 하나씩만 결과 출력
)

grad_cam = GradCam(model=model, blob_name = 'features', target_layer_names=['18'], use_cuda=True)
grad_cam_dann = GradCam(model=model_dann, blob_name = 'features', target_layer_names=['18'], use_cuda=True)
# img = cv2.imread('temp.jpg', 1)
# img = np.float32(cv2.resize(img, (224, 224))) / 255
# img = np.float32(img) / 255
# inputs = preprocess_image(img)
# If None, returns the map for the highest scoring category.
# Otherwise, targets the requested index.
for batch in test_dataset:
    inputs, targets = batch
    target_index = None
    mask_dic = grad_cam(inputs.unsqueeze(0), target_index)
    mask_dic_dann = grad_cam_dann(inputs.unsqueeze(0), target_index)

    for (_, mask), (_, mask_dann) in zip(mask_dic.items(), mask_dic_dann.items()):
        f, (ax1, ax2) = plt.subplots(1, 2)
        ax1.imshow(inputs.permute(1,2,0))
        cam = get_cam(mask)
        ax1.imshow(cam, alpha=0.7)
        
        ax2.imshow(inputs.permute(1,2,0))
        cam_dann = get_cam(mask_dann)
        ax2.imshow(cam_dann, alpha=0.7)
        
        plt.title(test_dataset.classes[targets])
        plt.savefig(f"output_img_tubuki/{target_dataset_name}_{test_dataset.classes[targets]}.png")
        plt.close()
None
# show_cams(inputs.permute(1,2,0), mask_dic, test_dataset.classes[targets])
# gb_model = GuidedBackpropReLUModel(model=model, activation_layer_name = 'ReLU6', use_cuda=False)
# show_gbs(inputs.unsqueeze(0), gb_model, target_index, mask_dic)